In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:

from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

MODEL_PATH = "/content/drive/MyDrive/ML_Models/weather_lstm_model.h5"
SCALER_PATH = "/content/drive/MyDrive/ML_Models/scaler.save"

# تحميل المودل والـ scaler مرة واحدة عند تشغيل السيرفر
model = load_model(MODEL_PATH, custom_objects={'mse': 'mse'})
scaler = joblib.load(SCALER_PATH)

# إعداد بعض المتغيرات (تأكد إنها متوافقة مع بياناتك)
n_past = 7  # عدد الأيام السابقة
features = ['SW_down', 'dewpoint_2m_C', 'temperature_2m_C']
n_features = len(features)
n_future = 90  # عدد الأيام المراد التنبؤ بها

# ---------------------------
# 3) إنشاء FastAPI app
# ---------------------------
app = FastAPI(title="Weather Forecast API")

class ForecastRequest(BaseModel):
    last_known_days: List[List[float]]  # آخر n_past يوم لكل feature
    n_days_forecast: int = 90           # عدد الأيام المراد التنبؤ بها

@app.post("/predict")
def predict(request: ForecastRequest):
    last_window = np.array(request.last_known_days)

    # التأكد من شكل الـ input
    if last_window.shape != (n_past, n_features):
        return {"error": f"last_known_days shape must be ({n_past}, {n_features})"}

    # تطبيع البيانات
    current_window = scaler.transform(last_window)
    predictions = []

    # تنبؤ متسلسل (recursive) لكل الأيام المطلوبة
    for _ in range(request.n_days_forecast):
        pred = model.predict(current_window.reshape(1, n_past, n_features), verbose=0)
        predictions.append(pred[0])
        # تحديث النافذة بإزالة أول يوم وإضافة اليوم المتوقع
        current_window = np.vstack([current_window[1:], pred])

    # تحويل القيم للحجم الأصلي
    predictions = scaler.inverse_transform(predictions)

    # تجهيز النتيجة
    forecast_df = pd.DataFrame(predictions, columns=features)
    return forecast_df.to_dict(orient='records')
